# FOR TESTING: Remove before use

In [1]:
%env AIBS_RIG_ID = NP.0

env: AIBS_RIG_ID=NP.1


***
# **Without mouse on stage**
## Restart kernel before running this cell

In [2]:
# %run pip install --extra-index-url https://pypi.org/simple  --upgrade --quiet np_workflows
%load_ext autoreload
%autoreload 2

import datetime
import pathlib
import time

import IPython
import ipywidgets as widgets

import np_logging
import np_services
import np_session
import np_workflows

from np_services import (
    Sync, VideoMVR, Cam3d, ScriptCamstim, NewScaleCoordinateRecorder, OpenEphys
)
from np_services.protocols import (
    Pretestable, Initializable, Startable, Testable, Stoppable, Finalizable, Pretestable, Verifiable, Validatable,
    TestError,
    )

logger = np_logging.getLogger()
logger.setLevel('INFO')

np_workflows.elapsed_time_widget()

ModuleNotFoundError: No module named 'np_config'

***
## Launch apps via RSC
[optional]

In [3]:
np_services.start_rsc_apps()

16:24 | np_config.np_config | DEBUG | Updated local config file C:\ProgramData\np\np_config\config_logs\zk_config-task_trained_network.2023-02-09_162420.yaml
16:24 | np_config.np_config | DEBUG | '/np_defaults/configuration' updated in local config file C:\ProgramData\np\np_config\config_logs\zk_config-task_trained_network.2023-02-09_162420.yaml
16:24 | np_config.np_config | DEBUG | Updated local config file C:\ProgramData\np\np_config\config_logs\zk_config-task_trained_network.2023-02-09_162420.yaml
16:24 | np_config.np_config | DEBUG | '/rigs/NP.1' updated in local config file C:\ProgramData\np\np_config\config_logs\zk_config-task_trained_network.2023-02-09_162420.yaml
16:24 | np_config.np_config | DEBUG | Updated local config file C:\ProgramData\np\np_config\config_logs\zk_config-task_trained_network.2023-02-09_162420.yaml
16:24 | np_config.np_config | DEBUG | '/projects/np_logging/defaults/configuration' updated in local config file C:\ProgramData\np\np_config\config_logs\zk_config

***
## Select mouse and user

In [4]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Check MTrain for mouse
[optional]

In [5]:
np_workflows.mtrain_widget(mouse)

***
## Generate new session and folder 

In [4]:
#! for testing
session = np_session.Session('1246096278_366122_20230209')
session_folder = session.npexp_path

In [6]:
session: np_session.Session = np_session.generate_session(mouse, user, session_type='ecephys')
display(session)
session_folder = session.npexp_path
session_folder.mkdir(parents=True, exist_ok=True)

Session('1246096278_366122_20230209')

***
## Setup components for the experiment

In [ ]:
services = (Sync, VideoMVR, Cam3d, OpenEphys, NewScaleCoordinateRecorder, ScriptCamstim)

***
## Pretest
### Checks before running

In [4]:
np_workflows.check_hardware_widget()

In [5]:
np_workflows.check_openephys_widget()

### Pretest critical components individually
Only proceed if all pretests pass!

In [ ]:
OpenEphys.pretest()

In [ ]:
VideoMVR.pretest()

In [ ]:
Cam3d.pretest()

In [ ]:
Sync.pretest()

In [ ]:
ScriptCamstim.pretest()

***
## Setup and reset all services

In [ ]:
ScriptCamstim.script = 'C:/ProgramData/StimulusFiles/dev/oct22_tt_stim_script.py'
ScriptCamstim.script = 'oct22_tt_stim_script.py' #! testing from notebook directory
ScriptCamstim.params = dict(
)

OpenEphys.folder = session_folder.name

NewScaleCoordinateRecorder.log_root = session_folder

for service in services:
    if isinstance(service, Initializable):
        service.initialize()
    if isinstance(service, Testable):
        service.test()

***
## Dip probes

In [8]:
np_workflows.dye_widget(session_folder)

In [ ]:
np_workflows.photodoc_widget('pre_experiment_surface_image')

***
***
# **With mouse on stage**

In [ ]:
np_workflows.check_mouse_widget()

NameError: name 'np_workflows' is not defined

***
## Lower cartridge

In [ ]:
np_workflows.photodoc_widget('brain_surface_image')

***
## Insert probes

In [ ]:
np_workflows.isi_widget(mouse.lims, colormap=False)

***
## Photodoc before advancing probes

In [ ]:
np_workflows.photodoc_widget('pre_insertion_surface_image')

***
## Settle timer

In [ ]:
np_workflows.print_countdown_timer(minutes=10)

***
## Start devices recording

In [ ]:
for service in (Sync, VideoMVR, OpenEphys):
    service.start()
    if isinstance(service, Verifiable):
        time.sleep(1)
        service.verify()

## Start experiment

In [ ]:
logger.info('Starting Camstim script')

ScriptCamstim.start()

# time.sleep(10)
# ScriptCamstim.verify() # needs updating to not check filesize

while not ScriptCamstim.is_ready_to_start():
    time.sleep(10)

if isinstance(ScriptCamstim, Finalizable):
    ScriptCamstim.finalize()

logger.info('Camstim script complete')

***
## Stop recording

In [ ]:
while not ScriptCamstim.is_ready_to_start():
    time.sleep(10)

for service in (_ for _ in (Sync, VideoMVR, OpenEphys) if isinstance(_, Stoppable)):
    service.stop()

for service in (_ for _ in (VideoMVR, OpenEphys, ScriptCamstim) if isinstance(_, Finalizable)): # Sync will be finalized later to avoid waiting on processing
    service.finalize()

for service in (_ for _ in (VideoMVR, OpenEphys, ScriptCamstim) if isinstance(_, Validatable)):
    service.validate()

***
## Before removing probes

In [ ]:
np_workflows.photodoc_widget('post_stimulus_surface_image')

***
## After fully retracting probes

In [ ]:
np_workflows.photodoc_widget('post_experiment_surface_image')

***
## After raising cartridge

In [ ]:
np_workflows.finishing_checks_widget()
np_workflows.copy_files(services, session_folder)